In [1]:
import pandas as pd

import myquery_db as query_db

In [2]:
tables = {'meta':'ks_project_yyk.ISRA_report.isra_result_meta',
          'detail':'ks_project_yyk.ISRA_report.isra_result_detail',
          'agg':'ks_project_yyk.ISRA_report.result_detail_agg',
          'imag_url':'ks_project_yyk.ISRA_report.image_url'
          }

In [ ]:
# main.py (FastAPI application)
import myquery_db as query_db
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from typing import List, Dict, Any
from datetime import datetime
import numpy as np
from fastapi.responses import JSONResponse
import pandas as pd





# 数据库连接字符串（根据你的实际情况修改）
# server: SQL Server 地址
# database: 数据库名
# trusted_connection: 信任连接（如果使用Windows身份验证）
# driver: ODBC 驱动名
tables = {'meta':'ks_project_yyk.ISRA_report.isra_result_meta',
          'detail':'ks_project_yyk.ISRA_report.isra_result_detail',
          'agg':'ks_project_yyk.ISRA_report.result_detail_agg',
          'imag_url':'ks_project_yyk.ISRA_report.result_meta_image_url'
          }

# 定义数据模型（对应前端的 TypeScript 类型）
# 在 Python 中，你可以用 Pydantic 来定义
from pydantic import BaseModel

class RawDataAll(BaseModel):
    id: str 
    roll_width_mm: int
    flaw_number: int
    roll_length_m: float
    lot: str
    line_speed_m_min: float
    reportdate: datetime
    product:str
    roll_number: str
    production_line: str
    country: str
    productid: int
    flaw_type: str
    flaw_area_mm2: float
    flaw_length_mm: float
    flaw_width_mm: float
    flaw_position_m: float
    image_url: str
    avg: float
    std: float

class RawDataMeta(BaseModel):
    #id: str 
    #roll_width_mm: int
    #flaw_number: int
    #roll_length_m: float
    #lot: str
    #line_speed_m_min: float
    reportdate: datetime
    product:str
    roll_number: str
    production_line: str
    #country: str


class OtherData(BaseModel):
    reportdate: datetime
    product:str
    roll_number: str
    lot: str
    flaw_type: str
    area_catogry: str
    area_range: str
    flaw_numbers: int


  # 定义一个新的模型来包装所有返回的数据
class DashboardResponse(BaseModel):
    raw_meta_data: List[RawDataAll]
    other_data: List[OtherData] # 假设这是你的 df2 对应的数据模型


def my_agg(df):
    # 定义你的阈值
    df['l1'] = df['avg'] - 3 * df['std']
    df['l2'] = df['avg'] - 2 * df['std']
    df['l3'] = df['avg'] - 1 * df['std']
    df['l5'] = df['avg'] + 1 * df['std']
    df['l6'] = df['avg'] + 2 * df['std']
    df['l7'] = df['avg'] + 3 * df['std']
    # 定义条件列表和对应的选择值列表
    print('1'*10)


   
    # 简化后的代码如下：
    conditions_simplified = [
        df['area'] > df['l7'],
        df['area'] > df['l6'],
        df['area'] > df['l5'],
        df['area'] > df['avg'],
        df['area'] > df['l3'],
        df['area'] > df['l2'],
        df['area'] > df['l1'],
         df['area'] >0
    ]

    choices_simplified = [
        "LEVEL8",
        "LEVEL7",
        "LEVEL6",
        "LEVEL5",
        "LEVEL4",
        "LEVEL3",
        "LEVEL2",
        "LEVEL1"
    ]

    df['area_catogary'] = np.select(conditions_simplified, choices_simplified, default="LEVEL1")
    print('2'*10,df.head())
    choices_range = [
    '(' + df['l7'].astype(str) + ', +∞)',
    '(' + df['l6'].astype(str) + ', ' + df['l7'].astype(str) + ']',
    '(' + df['l5'].astype(str) + ', ' + df['l6'].astype(str) + ']',
    '(' + df['avg'].astype(str) + ', ' + df['l5'].astype(str) + ']',
    '(' + np.maximum(df['l3'], 0).astype(str) + ', ' + df['avg'].astype(str) + ']',
    '(' + np.maximum(df['l2'], 0).astype(str) + ', ' + df['l3'].astype(str) + ']',
    '(' + np.maximum(df['l1'], 0).astype(str) + ', ' + df['l2'].astype(str) + ']',
    '(0, ' + df['l1'].astype(str) + ']'
        ]
    # 使用 np.select 创建新的列
    df['area_range'] = np.select(conditions_simplified, choices_range, default=None)
    print('3'*10,df.head())

    #groupy by:
    group_columns = ['reportdate','product','roll_number','lot','flaw_type','area_catogary','area_range']
    df_agg = df.groupby(group_columns).apply(lambda x: len(x))
    print('4'*10, df_agg.head())
    df_agg = df_agg.reset_index()
    df_agg.columns = group_columns+['flaw_numbers']
    return df_agg


def get_meta_data_1day(production_line):
    hour_gap = 9 if production_line=='K1' else 8
    hour_gap = hour_gap-70
    time_thresh = f"FORMAT(DATEADD(hour, {hour_gap}, GETUTCDATE()), 'yyyy-MM-dd')"
    results =  {'raw_meta_data':[],'other_data':[]}
    try:
        
        query = f'''SELECT d.*, 
            m.[roll_width_(mm)] as roll_width_mm,
            m.[roll_length_(m)] as roll_length_m,
            m.[line_speed_(m/min)] as line_speed_m_min,
            m.flaw_number, 
            m.lot, 
            m.reportdate, 
            m.product, 
            m.roll_number, 
            m.production_line, 
            m.country,
            u.image_url, 
            a.avg_area as avg, 
            a.std_area as std
        FROM {tables['detail']} d
            left join {tables['meta']} m on d.productid = m.id
            left join {tables['agg']} a on d.flaw_type = a.flaw_type
            left join {tables['imag_url']} u on u.flaw_detailid = d.id
        where 1=1
        and m.production_line ='{production_line}'
        and m.reportdate>=FORMAT(DATEADD(hour, {hour_gap}, GETUTCDATE()), 'yyyy-MM-dd')
        '''
        print(query)
        #df = query_db.query_ksdata(query)
        df = pd.read_csv('test.csv')

        df['reportdate'] = pd.to_datetime(df['reportdate'] ).dt.strftime('%Y-%m-%d %H:%M')
        print('df', df.head())
        df_agg = my_agg(df)
        print('df_agg', df_agg.head())
      
        results_df = df.to_dict(orient="records")
        results_agg = df_agg.to_dict(orient="records")
        results = {'raw_meta_data':results_df,'other_data':results_agg}
        #return JSONResponse(content=results)

        #return {'raw_meta_data':results_df,'other_data':results_agg}
    except Exception as e:
        print(f"Error connecting to database: {e}")
        
    return JSONResponse(content=results)
    


def get_meta_data():
    try:
        
        query = f'''SELECT
                        --[roll_width_(mm)] as roll_width_mm,
                        --flaw_number,
                        --[roll_length_(m)] as roll_length_m,
                        --lot,
                        --[line_speed_(m/min)] as line_speed_m_min,
                        reportdate,
                        product,
                        roll_number,
                        production_line
                        --country 
                    FROM {tables['meta']}
                    '''
        print(query)
        #df = query_db.query_ksdata(query)
        df = pd.read_csv('test_demo.csv')
        df['reportdate'] = pd.to_datetime(df['reportdate'] ).dt.strftime('%Y-%m-%d %H:%M')
        df['roll_number'] = df['roll_number'].astype(str)
        results = df.to_dict(orient="records")

        #return results
        return JSONResponse(content=results)
    except Exception as e:
        print(f"Error connecting to database: {e}")
        #return [{}]
        return JSONResponse(content=[{}])
   



class SelectedOptions(BaseModel):
    production_line: str
    reportdate: str
    product: str
    roll_number: str
    # Pydantic handles validation and type conversion automatically



def data_query_options(options: SelectedOptions):
    options_dict = options.model_dump()
    query = f'''SELECT d.*, 
            m.[roll_width_(mm)] as roll_width_mm,
            m.[roll_length_(m)] as roll_length_m,
            m.[line_speed_(m/min)] as line_speed_m_min,
            m.flaw_number, 
            m.lot, 
            m.reportdate, 
            m.product, 
            m.roll_number, 
            m.production_line, 
            m.country,
            u.image_url, 
            a.avg_area as avg, 
            a.std_area as std
        FROM {tables['detail']} d
            left join {tables['meta']} m on d.productid = m.id
            left join {tables['agg']} a on d.flaw_type = a.flaw_type
            left join {tables['imag_url']} u on u.flaw_detailid = d.id
        where 1=1
        '''
    options_dict_new = {}
    for k, v in options_dict:
        if v=='ALL' or v=='':
            continue
        if k=='roll_number':
            query += f' and {k}={v}'
  
        else:
            query += f''' and {k}='{v}' '''
    results = {'raw_meta_data':[{}],'other_data':[{}]}
    try:
        print(query)

        #df = query_db.query_ksdata(query)
        df = pd.read_csv('test.csv')
        df_agg = my_agg(df)
      
        results_df = df.to_dict(orient="records")
        results_agg = df_agg.to_dict(orient="records")

        results = {'raw_meta_data':results_df,'other_data':results_agg}
        #return JSONResponse(content=results)

        #return {'raw_meta_data':results_df,'other_data':results_agg}
    except Exception as e:
        print(f"Error connecting to database: {e}")
        
    return JSONResponse(content=results)
    


In [4]:
df = pd.read_csv('test.csv')



In [7]:
df.columns

Index(['id', 'productid', 'flaw_id', 'flaw_type', 'length', 'width', 'area',
       'downweb_position_(m)', 'crossweb_position_(mm)', 'roll_width_mm',
       'roll_length_m', 'line_speed_m_min', 'flaw_number', 'lot', 'reportdate',
       'product', 'roll_number', 'production_line', 'country', 'image_url',
       'avg', 'std', 'l1', 'l2', 'l3', 'l5', 'l6', 'l7', 'area_catogary',
       'area_range'],
      dtype='object')

In [9]:
df['l1'] = df['avg'] - 3 * df['std']
df['l2'] = df['avg'] - 2 * df['std']
df['l3'] = df['avg'] - 1 * df['std']
df['l5'] = df['avg'] + 1 * df['std']
df['l6'] = df['avg'] + 2 * df['std']
df['l7'] = df['avg'] + 3 * df['std']

In [10]:
conditions_simplified = [
        df['area'] > df['l7'],
        df['area'] > df['l6'],
        df['area'] > df['l5'],
        df['area'] > df['avg'],
        df['area'] > df['l3'],
        df['area'] > df['l2'],
        df['area'] > df['l1'],
         df['area'] >0
    ]

choices_simplified = [
        "LEVEL8",
        "LEVEL7",
        "LEVEL6",
        "LEVEL5",
        "LEVEL4",
        "LEVEL3",
        "LEVEL2",
        "LEVEL1"
    ]

In [11]:
df['area_catogary'] = np.select(conditions_simplified, choices_simplified, default="LEVEL1")


In [12]:

choices_range = [
'(' + df['l7'].astype(str) + ', +∞)',
'(' + df['l6'].astype(str) + ', ' + df['l7'].astype(str) + ']',
'(' + df['l5'].astype(str) + ', ' + df['l6'].astype(str) + ']',
'(' + df['avg'].astype(str) + ', ' + df['l5'].astype(str) + ']',
'(' + np.maximum(df['l3'], 0).astype(str) + ', ' + df['avg'].astype(str) + ']',
'(' + np.maximum(df['l2'], 0).astype(str) + ', ' + df['l3'].astype(str) + ']',
'(' + np.maximum(df['l1'], 0).astype(str) + ', ' + df['l2'].astype(str) + ']',
'(0, ' + df['l1'].astype(str) + ']'
    ]

In [13]:
df['area_range'] = np.select(conditions_simplified, choices_range, default=None)


In [14]:
df.head()

,id,productid,flaw_id,flaw_type,length,width,area,downweb_position_(m),crossweb_position_(mm),roll_width_mm,...,avg,std,l1,l2,l3,l5,l6,l7,area_catogary,area_range
0,744481,330,3,splice,379.388,499.027,189325.047,0.200,33.9,1058.81,...,163080.17868,183571.299361,-387633.719403,-204062.420042,-20491.120681,346651.478041,530222.777402,713794.076763,LEVEL5,"(163080.17867998983, 346651.47804108664]"
1,744482,330,4,splice,379.388,499.027,189325.047,0.546,33.9,1058.81,...,163080.17868,183571.299361,-387633.719403,-204062.420042,-20491.120681,346651.478041,530222.777402,713794.076763,LEVEL5,"(163080.17867998983, 346651.47804108664]"
2,744483,330,5,splice,379.388,499.027,189325.047,0.892,33.9,1058.81,...,163080.17868,183571.299361,-387633.719403,-204062.420042,-20491.120681,346651.478041,530222.777402,713794.076763,LEVEL5,"(163080.17867998983, 346651.47804108664]"
3,744484,330,6,splice,379.388,499.027,189325.047,1.238,33.9,1058.81,...,163080.17868,183571.299361,-387633.719403,-204062.420042,-20491.120681,346651.478041,530222.777402,713794.076763,LEVEL5,"(163080.17867998983, 346651.47804108664]"
4,744485,330,7,splice,379.388,499.027,189325.047,1.584,33.9,1058.81,...,163080.17868,183571.299361,-387633.719403,-204062.420042,-20491.120681,346651.478041,530222.777402,713794.076763,LEVEL5,"(163080.17867998983, 346651.47804108664]"


In [17]:
df.columns

Index(['id', 'productid', 'flaw_id', 'flaw_type', 'length', 'width', 'area',
       'downweb_position_(m)', 'crossweb_position_(mm)', 'roll_width_mm',
       'roll_length_m', 'line_speed_m_min', 'flaw_number', 'lot', 'reportdate',
       'product', 'roll_number', 'production_line', 'country', 'image_url',
       'avg', 'std', 'l1', 'l2', 'l3', 'l5', 'l6', 'l7', 'area_catogary',
       'area_range'],
      dtype='object')

In [22]:
df['lot'] = df['lot'].fillna('none')

In [23]:

#groupy by:
group_columns = ['reportdate','product','roll_number','lot','flaw_type','area_catogary','area_range']
df_agg = df.groupby(group_columns).apply(lambda x: len(x))
df_agg

C:\Users\0185972\AppData\Local\Temp\ipykernel_21968\1976391257.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_agg = df.groupby(group_columns).apply(lambda x: len(x))


reportdate               product    roll_number  lot   flaw_type           area_catogary  area_range                               
2025-09-10 00:12:01.000  COC-White  3250909028   none  Face Large Black    LEVEL4         (0.0, 249.772717274961]                      18
                                                                           LEVEL5         (249.772717274961, 2247.5464788624395]       11
                                                       Face Small Black    LEVEL3         (0.13489423036975123, 0.3154758360902816]     2
                                                       splice              LEVEL4         (0.0, 163080.17867998983]                     4
                                                                           LEVEL5         (163080.17867998983, 346651.47804108664]     15
                                                                                                                                       ..
2025-09-10 13:45:53.000  Film-G     3250

In [24]:
df_agg

reportdate               product    roll_number  lot   flaw_type           area_catogary  area_range                               
2025-09-10 00:12:01.000  COC-White  3250909028   none  Face Large Black    LEVEL4         (0.0, 249.772717274961]                      18
                                                                           LEVEL5         (249.772717274961, 2247.5464788624395]       11
                                                       Face Small Black    LEVEL3         (0.13489423036975123, 0.3154758360902816]     2
                                                       splice              LEVEL4         (0.0, 163080.17867998983]                     4
                                                                           LEVEL5         (163080.17867998983, 346651.47804108664]     15
                                                                                                                                       ..
2025-09-10 13:45:53.000  Film-G     3250

In [25]:


print('4'*10, df_agg.head())
df_agg = df_agg.reset_index()
df_agg.columns = group_columns+['flaw_numbers']


4444444444 reportdate               product    roll_number  lot   flaw_type         area_catogary  area_range                               
2025-09-10 00:12:01.000  COC-White  3250909028   none  Face Large Black  LEVEL4         (0.0, 249.772717274961]                      18
                                                                         LEVEL5         (249.772717274961, 2247.5464788624395]       11
                                                       Face Small Black  LEVEL3         (0.13489423036975123, 0.3154758360902816]     2
                                                       splice            LEVEL4         (0.0, 163080.17867998983]                     4
                                                                         LEVEL5         (163080.17867998983, 346651.47804108664]     15
dtype: int64


In [26]:
df_agg

,reportdate,product,roll_number,lot,flaw_type,area_catogary,area_range,flaw_numbers
0,2025-09-10 00:12:01.000,COC-White,3250909028,none,Face Large Black,LEVEL4,"(0.0, 249.772717274961]",18
1,2025-09-10 00:12:01.000,COC-White,3250909028,none,Face Large Black,LEVEL5,"(249.772717274961, 2247.5464788624395]",11
2,2025-09-10 00:12:01.000,COC-White,3250909028,none,Face Small Black,LEVEL3,"(0.13489423036975123, 0.3154758360902816]",2
3,2025-09-10 00:12:01.000,COC-White,3250909028,none,splice,LEVEL4,"(0.0, 163080.17867998983]",4
4,2025-09-10 00:12:01.000,COC-White,3250909028,none,splice,LEVEL5,"(163080.17867998983, 346651.47804108664]",15
...,...,...,...,...,...,...,...,...
343,2025-09-10 13:45:53.000,Film-G,3250910007,none,Liner Medium black,LEVEL3,"(0.2839174057174789, 1.250870606495959]",2
344,2025-09-10 13:45:53.000,Film-G,3250910007,none,Liner Medium black,LEVEL4,"(1.250870606495959, 2.217823807274439]",15
345,2025-09-10 13:45:53.000,Film-G,3250910007,none,Liner Medium black,LEVEL5,"(2.217823807274439, 3.1847770080529187]",13
346,2025-09-10 13:45:53.000,Film-G,3250910007,none,Liner Medium black,LEVEL6,"(3.1847770080529187, 4.151730208831399]",2


In [6]:
df_agg = my_agg(df)

1111111111
2222222222        id  productid  flaw_id flaw_type   length    width        area  \
0  744481        330        3    splice  379.388  499.027  189325.047   
1  744482        330        4    splice  379.388  499.027  189325.047   
2  744483        330        5    splice  379.388  499.027  189325.047   
3  744484        330        6    splice  379.388  499.027  189325.047   
4  744485        330        7    splice  379.388  499.027  189325.047   

   downweb_position_(m)  crossweb_position_(mm)  roll_width_mm  ...  \
0                 0.200                    33.9        1058.81  ...   
1                 0.546                    33.9        1058.81  ...   
2                 0.892                    33.9        1058.81  ...   
3                 1.238                    33.9        1058.81  ...   
4                 1.584                    33.9        1058.81  ...   

                                           image_url           avg  \
0  https://10.161.67.41:8443/korean_api/im

C:\Users\0185972\AppData\Local\Temp\ipykernel_21968\1462925005.py:138: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_agg = df.groupby(group_columns).apply(lambda x: len(x))


ValueError: cannot insert area_range, already exists